* In `BusinessRules.csv` staan twee voorbeelden van multi-line teksten:
  * Regel 1379: `BusinessRule` met `ID` 1379 heeft een `Description` met een newline, maar met een correcte delimiters (")
  * Regel 9263: `BusinessRule` met `ID` 9263 heeft een `Description` met een newline, maar met een incorrecte delimiters (geen ")

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('BusinessRules.csv')


In [8]:
df.columns

Index(['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'VersionNum',
       'ImportDate', 'IssuerId2', 'TIN', 'ProductId', 'StandardComponentId',
       'EnrolleeContractRateDeterminationRule',
       'TwoParentFamilyMaxDependentsRule',
       'SingleParentFamilyMaxDependentsRule', 'DependentMaximumAgRule',
       'ChildrenOnlyContractMaxChildrenRule',
       'DomesticPartnerAsSpouseIndicator', 'SameSexPartnerAsSpouseIndicator',
       'AgeDeterminationRule', 'MinimumTobaccoFreeMonthsRule',
       'CohabitationRule', 'RowNumber', 'MarketCoverage', 'DentalOnlyPlan'],
      dtype='object')

In [28]:
# Regel 1379 levert geen problemen op, de '\n' is keurig opgenomen:
df.loc[[1375,1376,1377,1378,1379,1380],["EnrolleeContractRateDeterminationRule"]].values

array([['A different rate (specifically for parties of two or more)for each enrollee is added together'],
       ['A different rate (specifically for parties of two or more)for each enrollee is added together'],
       ['A different rate (specifically for parties of two or more)\nfor each enrollee is added together'],
       ['A different rate (specifically for parties of two or more)for each enrollee is added together'],
       ['A different rate (specifically for parties of two or more)for each enrollee is added together'],
       ['A different rate (specifically for parties of two or more)for each enrollee is added together']],
      dtype=object)

In [31]:
# Regel 9263/9264 levert wel uitdagingen op:
df.loc[[9260,9261,9262,9263,9264,9265],["EnrolleeContractRateDeterminationRule"]]

,EnrolleeContractRateDeterminationRule
9260,A different rate (specifically for parties of ...
9261,10
9262,A different rate (specifically for parties of ...
9263,A different rate (specifically for parties of ...
9264,A different rate (specifically for parties of ...
9265,A different rate (specifically for parties of ...


In [33]:
df.loc[[9260,9261,9262]]

,BusinessYear,StateCode,IssuerId,SourceName,VersionNum,ImportDate,IssuerId2,TIN,ProductId,StandardComponentId,...,DependentMaximumAgRule,ChildrenOnlyContractMaxChildrenRule,DomesticPartnerAsSpouseIndicator,SameSexPartnerAsSpouseIndicator,AgeDeterminationRule,MinimumTobaccoFreeMonthsRule,CohabitationRule,RowNumber,MarketCoverage,DentalOnlyPlan
9260,2015,IL,68432,SERFF,6,2015-02-22 21:18:17,68432,27-2186150,68432IL003,68432IL0030004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9261,for each enrollee is added together,3 or more,3 or more,25,1,Yes,Yes,Age on effective date,6,"Spouse,No;Adopted Child,No;Foster Child,No;War...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9262,2015,IL,68432,SERFF,6,2015-02-22 21:18:17,68432,27-2186150,68432IL003,68432IL0030005,...,25,1,Yes,Yes,Age on effective date,6,"Spouse,No;Adopted Child,No;Foster Child,No;War...",10.0,Individual,No


Het inlezen gaat hier wel goed, alleen de datatypes kloppen niet (pandas speelt op safe en dwingt geen getal af voor BusinessYear). Bij omzetten van datatype speelt dit wel op:

In [37]:
# Levert een fout op voor regel 9261, waar de kolom BusinessYear de waarde 'for each employee is added together' bevat:
try:
    df["BusinessYear"].astype(int)
except ValueError as e:
    print(f"Error: {e}")

Error: invalid literal for int() with base 10: 'for each enrollee is added together'


De oplossing hier is om een stukje logica op te nemen die het volgende doet:

* Kijk naar een zuiver numerieke kolom die altijd gevuld is (zoals hier BusinessYear)
* Zoek de niet-numerieke waarden
  * Dit is de rij die een foutmelding geeft
* Achterhaal in welke kolom de data is "afgebroken"
* Vul de voorliggende rij aan met de data van de "foute" rij

Deze methode is niet waterdicht: er kan in een ander veld ook een newline voorkomen die de kolom BusinessYear wél een geldige numerieke waarde geeft, maar nog steeds incorrect is afgebroken. Je hebt hier waarschijnlijk echter al de eerste 80% te pakken en kunt op dit patroon verderbouwen voor andere uizonderingen.

Dit kán in pandas, maar is efficiënter om als pre-processing te doen:

In [93]:
import io

# Read the file and clean it up
def read_and_clean_csv(file_path):
    # We use a StringIO object to store the cleaned up CSV file
    # In this way, we will not alter the original file
    # However, it might consume somewhat more memory - you might want to use a temporary file instead
    cleaned_csv = io.StringIO()
    
    with open(file_path, 'r', encoding='utf-8') as f:
        prev_line = f.readline()
        for line in f:
            if not line.split(',')[0].isnumeric():
                prev_line = prev_line.strip() + ' ' + line.strip()
            else:
                cleaned_csv.write(prev_line + '\n')
                prev_line = line
    
    cleaned_csv.write(prev_line + '\n')
    cleaned_csv.seek(0)
    return pd.read_csv(cleaned_csv)

file_path = 'BusinessRules.csv'
df = read_and_clean_csv(file_path)

In [95]:
# Nu gaat dit ook goed:
try:
    df["BusinessYear"].astype(int)
except ValueError as e:
    print(f"Error: {e}")